### a. Data cleaning


In [88]:
import pandas as pd
import seaborn as sns

In [89]:
df = pd.read_csv("Heart.csv")
df.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [90]:
df = df.drop('Unnamed: 0',axis=1).head()

In [64]:
df.isnull().sum() 

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         0
AHD          0
dtype: int64

In [65]:
med = df['Age'].mean()
med

55.0

In [66]:
df['Age'] = df['Age'].fillna(med,inplace=False)

In [67]:
print(df.isnull().sum())

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         0
AHD          0
dtype: int64


### b. Data integration

In [8]:
import sqlite3
db = sqlite3.connect('sample.db',timeout=20)
c = db.cursor()

In [9]:
c.execute('create table company2(id int, name string, age int, salary int)')

In [10]:
c.execute('insert into company2 values(1,"rohan",21, 10000)')
c.execute('insert into company2 values(2,"rohit",20, 15000)')

In [11]:
data = c.execute("select * from company2")
for i in data:
    print(i)

(1, 'rohan', 21, 10000)
(2, 'rohit', 20, 15000)


In [12]:
c.execute('DROP TABLE company2')

### d. Error correcting

In [68]:
df = df.dropna(subset=['Thal'])
df.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


### e. Data model building

In [69]:
df.corr()


,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca
Age,1.000000,0.530060,0.389432,0.384004,0.302891,0.681505,-0.920032,0.741929,-0.294651,0.080951,0.718370
Sex,0.530060,1.000000,0.250000,0.670752,0.250000,-0.250000,-0.400318,0.408248,0.557736,0.801784,0.395285
RestBP,0.389432,0.250000,1.000000,0.721821,0.285714,0.250000,-0.512688,0.174964,-0.474354,0.152721,0.395285
Chol,0.384004,0.670752,0.721821,1.000000,-0.136362,-0.176902,-0.549089,0.514566,0.041875,0.380201,0.664301
Fbs,0.302891,0.250000,0.285714,-0.136362,1.000000,0.250000,0.014046,-0.408248,0.025941,0.534522,-0.395285
RestECG,0.681505,-0.250000,0.250000,-0.176902,0.250000,1.000000,-0.663685,0.408248,-0.804178,-0.534522,0.395285
MaxHR,-0.920032,-0.400318,-0.512688,-0.549089,0.014046,-0.663685,1.000000,-0.880224,0.471322,0.148284,-0.910572
ExAng,0.741929,0.408248,0.174964,0.514566,-0.408248,0.408248,-0.880224,1.000000,-0.222400,-0.218218,0.968246
Oldpeak,-0.294651,0.557736,-0.474354,0.041875,0.025941,-0.804178,0.471322,-0.222400,1.000000,0.741840,-0.328134
Slope,0.080951,0.801784,0.152721,0.380201,0.534522,-0.534522,0.148284,-0.218218,0.741840,1.000000,-0.211289


In [72]:
x = df[['Ca']]
y = df['ExAng']

In [73]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0)

In [74]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm

LinearRegression()

In [75]:
lm.fit(x_train,y_train)
predicted = lm.predict(x_train)

In [76]:
y_intercept = lm.intercept_
slope_m = lm.coef_

In [77]:
lm.score(x_test,y_test)

0.7777777777777777

In [84]:
print(lm.predict([[56]]))

[18.66666667]


In [85]:
def prediction(y_intercept,slope_m,x):
    y = slope_m*x+y_intercept
    return y

In [86]:
print(prediction(y_intercept,slope_m,56))

[18.66666667]


# Thanks Done this Assignment 